# Routing stack test

* Date: 2023-01-05
* Issue: <https://github.com/WasteLabs/streamlit_example_routing_app/issues/1>

## Scope

Test the following routing stack:

* <https://openrouteservice.org/dev/#/api-docs>: for the routing
* <https://github.com/gis-ops/routingpy>: to set it up
* <https://github.com/VROOM-Project/pyvroom>: for routing optimisation

## Imports and jupyter setup

In [1]:
# Imports / Configs / Global vars

# Import of native python tools
import os
import json
from functools import reduce

# Import of base ML stack libs
import numpy as np
import sklearn as sc

# Logging configuraiton
import logging
logging.basicConfig(format='[ %(asctime)s ][ %(levelname)s ]: %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
logger = logging.getLogger()
logger.setLevel(logging.INFO)


# Ipython configs
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
display(HTML("<style>.container { width:100% !important; }</style>"))
InteractiveShell.ast_node_interactivity = 'all'

# Pandas configs
import pandas as pd
pd.options.display.max_rows = 350
pd.options.display.max_columns = 250

# Jupyter configs
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

# GLOBAL VARS
from pathlib import Path
import sys
PROJECT_DIR = os.getcwd().rsplit('/', 1)[0]

if PROJECT_DIR not in sys.path:
    sys.path.append(PROJECT_DIR)

/var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_35529/1677521232.py:20: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## Test operouteservice and routingpy

Linked with <https://github.com/WasteLabs/streamlit_example_routing_app/issues/2>, we get a local instance running at localhost <http://localhost:8080/>.

This requires a whole setup separate setup.


### Get test data



In [2]:
import plotly.express as px


def load_transform_data():
    """Load data and do some basic transformation. The `st.experimental_singleton`
    decorator prevents the data from being continously reloaded.
    """
    RENAME_COLUMNS = {
        "car_hours": "demand",
        "centroid_lat": "latitude",
        "centroid_lon": "longitude",
    }
    COLUMN_ORDER = [
        "route_id",
        "stop_id",
        "demand",
        "duration",
        "latitude",
        "longitude",
    ]
    data = px.data.carshare()
    data = (
        data.assign(
            route_id=data["peak_hour"],
            duration=data.index,
        )
        .sort_values(["route_id"])
        .assign(stop_id=np.arange(0, data.shape[0]))
        .rename(columns=RENAME_COLUMNS)
    )[COLUMN_ORDER]
    skill_mapping = {"cycling-electric": range(0, 10), "driving-car": range(10, data.shape[0])}
    for skill in skill_mapping:
        data.loc[data["route_id"].isin(skill_mapping[skill]), "skill"] = skill

    return data

In [3]:
stop_data = load_transform_data()
stop_data

,route_id,stop_id,demand,duration,latitude,longitude,skill
121,0,0,1414.916667,121,45.485231,-73.617825,cycling-electric
223,0,1,1019.750000,223,45.573420,-73.632773,cycling-electric
74,0,2,1322.333333,74,45.607516,-73.513756,cycling-electric
167,0,3,342.666667,167,45.570596,-73.528281,cycling-electric
70,0,4,627.416667,70,45.573831,-73.604143,cycling-electric
102,0,5,1315.666667,102,45.596546,-73.525934,cycling-electric
65,0,6,1019.000000,65,45.514053,-73.632823,cycling-electric
18,0,7,670.750000,18,45.609668,-73.540232,cycling-electric
61,0,8,630.750000,61,45.542824,-73.539783,cycling-electric
192,0,9,538.750000,192,45.525676,-73.590037,cycling-electric


In [27]:
from routingpy import ORS

test_route = stop_data.iloc[:10].apply(lambda x: [x["longitude"], x["latitude"]], axis=1).tolist()
test_route

[[-73.61782510513464, 45.48523078626532],
 [-73.6327727768197, 45.57342046906779],
 [-73.51375636627016, 45.60751580853944],
 [-73.52828137402979, 45.570596118677166],
 [-73.60414320228709, 45.57383095629989],
 [-73.52593434063824, 45.59654571636812],
 [-73.6328226471968, 45.51405331206955],
 [-73.54023161013792, 45.609667861846745],
 [-73.53978288809766, 45.54282395560971],
 [-73.59003684764208, 45.52567648569128]]

In [34]:

client = ORS(base_url='http://localhost:8080/ors')
route = client.directions(
    locations = test_route,
    profile='driving-car',
    format="geojson",
    dry_run=False,
    attributes=["avgspeed", "detourfactor", "percentage"]
)

In [46]:
route.distance
route.duration
route.raw.keys()

104589

9832

dict_keys(['type', 'metadata', 'features', 'bbox'])